In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

2024-05-15 02:16:32.499667: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 02:16:33.175256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.16.1


In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-05-15 02:16:33.738712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 02:16:33.767275: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 02:16:33.767323: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import keras
from keras import layers
import zipfile 
from docx import Document
from PyPDF2 import PdfReader
import textract
import os
from docx import Document
import subprocess
#import win32com.client This is for Windows application.

/home/viky/miniconda3/envs/patents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# paths 
tfrecord_file = '/mnt/f/tfrecords/Dataset_9.tfrecord'
patent_folder = '/mnt/e/Walsh_work/Google_patent_Downloads/1)Patent_pdf'
standard_folder = '/mnt/d/3gpp_Standards'
extraction_dir = 'extracted_files'

In [5]:
# Load pre-trained BERT model and tokenizer
bert_model_name = 'bert-base-uncased'
bert_model = TFAutoModel.from_pretrained(bert_model_name)
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

/home/viky/miniconda3/envs/patents/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-15 02:16:40.680692: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 02:16:40.680794: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 02:16:40.680894: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA su

In [6]:
# Define function to encode text using BERT
def encode_text(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='tf')
    return bert_model(input_ids)[0][:, 0, :]  # Extract CLS token embedding

In [8]:
def create_model():
    patent_input = tf.keras.layers.Input(shape=(768,), name='patent_input')
    standard_input = tf.keras.layers.Input(shape=(768,), name='standard_input')
    
    concatenated = tf.keras.layers.Concatenate()([patent_input, standard_input])
    x = tf.keras.layers.Dense(512, activation='relu')(concatenated)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[patent_input, standard_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model()
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 patent_input (InputLayer)   [(None, 768)]                0         []                            
                                                                                                  
 standard_input (InputLayer  [(None, 768)]                0         []                            
 )                                                                                                
                                                                                                  
 concatenate_1 (Concatenate  (None, 1536)                 0         ['patent_input[0][0]',        
 )                                                                   'standard_input[0][0]']      
                                                                                            

In [9]:
def extract_patent_text(patent_file):
    patent_text = ""
    try:
        with open(patent_file, 'rb') as file:
            pdf_reader = PdfReader(file)
            for page in pdf_reader.pages:
                page_text = page.extract_text()
                if page_text:
                    patent_text += page_text
    except Exception as e:
        print(f"Error reading {patent_file}: {e}")
    return patent_text

In [10]:
def extract_docx_text(docx_file):
    try:
        doc = Document(docx_file)
        return '\n'.join([paragraph.text for paragraph in doc.paragraphs])
    except Exception as e:
        print(f"Exception during DOCX text extraction: {e}")
        return ""

def extract_doc_text(doc_file):
    try:
        # Use antiword to convert .doc to text
        command = f"antiword '{doc_file}'"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            return result.stdout
        else:
            raise Exception(f"antiword failed: {result.stderr}")
    except Exception as e:
        print(f"Exception during DOC text extraction: {e}")
        return ""

def extract_text(file_path):
    if file_path.endswith('.docx'):
        return extract_docx_text(file_path)
    elif file_path.endswith('.doc'):
        return extract_doc_text(file_path)
    else:
        return ""

def extract_standard_text(standard_zip, extraction_path):
    standard_text = ""
    extraction_subdir = os.path.join(extraction_path, os.path.basename(standard_zip).replace('.zip', ''))
    os.makedirs(extraction_subdir, exist_ok=True)
    
    try:
        with zipfile.ZipFile(standard_zip, 'r') as zip_ref:
            zip_ref.extractall(extraction_subdir)
            for root, _, files in os.walk(extraction_subdir):
                for file in files:
                    file_path = os.path.join(root, file)
                    text = extract_text(file_path)
                    standard_text += text
    except Exception as e:
        print(f"Error extracting from {standard_zip}: {e}")
    
    return standard_text

### Lets make the below cell as a debug cell for text extraction purposes. 

In [11]:
# if __name__ == "__main__":
#     # Example paths (ensure these paths are correct)
#     patent_example_path = '/mnt/e/walsh_work/Google_patent_Downloads/1)Patent_pdf/US9036607.pdf'
#     standard_example_zip = '/mnt/d/3gpp_Standards/TS 36.306 v10.15.0.zip'
#     extraction_dir = 'extracted_files'

#     # Extract and print patent text
#     patent_text = extract_patent_text(patent_example_path)
#     print("Extracted Patent Text:", patent_text)

#     # Extract and print standard text
#     standard_text = extract_standard_text(standard_example_zip, extraction_dir)
#     print("\nExtracted Standard Text:\n", standard_text)

In [12]:
def parse_tfrecord(example):
    feature_description = {
        'patent': tf.io.FixedLenFeature([], tf.string),
        'standard': tf.io.FixedLenFeature([], tf.string),
        'match': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example['patent'], example['standard'], example['match']
    

In [ ]:
def parse_tfrecord(record):
    feature_description = {
        'standard': tf.io.FixedLenFeature([], tf.string),
        'patent': tf.io.FixedLenFeature([], tf.string),
        'match': tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(record, feature_description)
    return example['standard'].numpy().decode(), example['patent'].numpy().decode(), example['match'].numpy()


# Create a TFRecordDataset to read the file
dataset = tf.data.TFRecordDataset(tfrecord_file)

# Iterate over the records and print their contents
for record in dataset:
    standard, patent, match = parse_tfrecord(record)
    print(f"Standard: {standard}")
    print(f"Patent: {patent}")
    print(f"Match: {match}")
    print("---")

Standard: TS 38.331 v17.5.0.zip
Patent: US20180279326A1
Match: 0
---
Standard: TS 38.211 v17.5.0.zip
Patent: US20200068465A1
Match: 0
---
Standard: TS 38.213 v17.6.0.zip
Patent: US20140192651A1
Match: 0
---
Standard: TS 38.321 v17.5.0.zip
Patent: US20200389887A1
Match: 1
---
Standard: TS 38.214 v17.6.0.zip
Patent: US9948378
Match: 0
---
Standard: TS 36.101 v14.2.1.zip
Patent: US9992812
Match: 0
---
Standard: TS 23.501 (17.9.0).zip
Patent: US20220053442A1
Match: 0
---
Standard: TS 38.300 v17.5.0.zip
Patent: US9642119
Match: 0
---
Standard: TS 123 287 v16.5.0.zip
Patent: US20020034959A1
Match: 1
---
Standard: TS 36.331 v9.9.0.zip
Patent: US8325860
Match: 1
---
Standard: TS 38.212 v17.5.0.zip
Patent: US20190181983A1
Match: 1
---
Standard: TS 36.211 v9.1.0.zip
Patent: US20100246712A1
Match: 1
---
Standard: TS 25.321 (9.5.0).zip
Patent: US20100020772A1
Match: 1
---
Standard: TS 25.214 v6.7.0.zip
Patent: US9282475
Match: 1
---
Standard: TS 38.213 v17.6.0.zip
Patent: US8812010
Match: 0
---
St

### In my TensorFlow dataset pipeline, when I am using tf.py_function to integrate non-TensorFlow functions (like BERT tokenizer from Hugging Face), we need to ensure that the text inputs are converted from tensors to strings before they are passed to the tokenizer. Let’s correct this in our prepare_data function:

### The error we are encountering occurs because the tf.py_function used in our preprocessing function tries to convert a TensorFlow tensor to a NumPy array inside a graph execution context, where tensors are symbolic and do not have direct access to their values. The .numpy() method is only available on EagerTensor objects when running TensorFlow in eager execution mode, not within a graph or a model's layer.

### To resolve this issue, we will need to restructure how you handle the preprocessing so that it is compatible with TensorFlow's graph execution. 

### Approach 1: Moving Text Processing Out of the Model

### One way to handle this is to preprocess the data entirely outside the model, converting text to embeddings before it even reaches the model's input. 

In [13]:
def preprocess_text(text):
    text = text.numpy().decode('utf-8')
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    output = bert_model(encoded['input_ids'], attention_mask=encoded['attention_mask'])
    return output.last_hidden_state[:, 0, :]  # Getting the CLS token embedding

def prepare_data(patent, standard, match):
    patent_embedding, standard_embedding = tf.py_function(
        lambda x, y: (preprocess_text(x), preprocess_text(y)), 
        [patent, standard], 
        [tf.float32, tf.float32]
    )
    match = tf.reshape(match, [-1])
    # Ensure the shapes are set correctly for both embeddings
    patent_embedding.set_shape([None, 768])
    standard_embedding.set_shape([None, 768])
    
    return {'patent_input': patent_embedding, 'standard_input': standard_embedding}, match

In [14]:
def load_and_prepare_dataset(file_path, batch_size=32):
    dataset = tf.data.TFRecordDataset(file_path)

    # Apply the parse_tfrecord to extract the data
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)

    # Apply the preprocess and reshape function
    dataset = dataset.map(prepare_data, num_parallel_calls=tf.data.AUTOTUNE)

    # Batch the dataset
    dataset = dataset.batch(batch_size)

    # Prefetch to improve speed of input pipeline
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [15]:
batch_size = 32
dataset = load_and_prepare_dataset(tfrecord_file, batch_size=batch_size)

In [ ]:
def inspect_dataset(dataset, num_examples=5):
    for features, labels in dataset.take(num_examples):
        print("Features:", features)  # Print feature details like shapes and types
        print("Labels:", labels)  # Print label details
        print("\n")

# Assuming train_dataset and validation_dataset are already defined
print("Inspecting Training Dataset")
inspect_dataset(train_dataset)

print("Inspecting Validation Dataset")
inspect_dataset(validation_dataset)

In [16]:
# Train the model
history = model.fit(dataset, epochs=10)

Epoch 1/10


I0000 00:00:1715753855.906987  213334 service.cc:145] XLA service 0x7f4405a24800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715753855.907034  213334 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-05-15 02:17:35.931962: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-15 02:17:35.988845: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1715753856.084498  213334 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


     43/Unknown - 634s 14s/step - loss: 0.7604 - accuracy: 0.4993

2024-05-15 02:27:52.666634: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Input to reshape is a tensor with 1572864 values, but the requested shape has 393216
2024-05-15 02:27:52.666678: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 7425769224486107554
2024-05-15 02:27:52.666704: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Input to reshape is a tensor with 1572864 values, but the requested shape has 393216
2024-05-15 02:27:52.702351: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: InvalidArgumentError: Exception encountered when calling layer 'dense' (type Dense).

{{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:GPU:0}} Input to reshape is a tensor with 1572864 values, but the requested shape has 393216 [Op:Reshape]

Call arguments received by layer 'dense' 

     44/Unknown - 649s 14s/step - loss: 0.7580 - accuracy: 0.5036

2024-05-15 02:27:56.477542: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: UNKNOWN: InvalidArgumentError: Exception encountered when calling layer 'key' (type Dense).

{{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:GPU:0}} Input to reshape is a tensor with 1572864 values, but the requested shape has 393216 [Op:Reshape]

Call arguments received by layer 'key' (type Dense):
  • inputs=tf.Tensor(shape=(1, 512, 768), dtype=float32)
Traceback (most recent call last):

  File "/home/viky/miniconda3/envs/patents/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/viky/miniconda3/envs/patents/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/viky/miniconda3/en

UnknownError: Graph execution error:

Detected at node EagerPyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  InvalidArgumentError: Exception encountered when calling layer 'key' (type Dense).

{{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:GPU:0}} Input to reshape is a tensor with 1572864 values, but the requested shape has 393216 [Op:Reshape]

Call arguments received by layer 'key' (type Dense):
  • inputs=tf.Tensor(shape=(1, 512, 768), dtype=float32)
Traceback (most recent call last):

  File "/home/viky/miniconda3/envs/patents/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/viky/miniconda3/envs/patents/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^